# Jimmy Wrangler Project

In the face of climate change and global warming, society today is more obsessed than ever about taking care of the environment. One way that people have begun to take steps in their everyday lives to cut down on their CO2 emissions is by switching their vehicle of commute from automobiles to bicycles.

According to a Statista statistic ([Click here](https://www.statista.com/statistics/740346/global-bicycle-market-size/)), the global bicycle market is expected to grow by a factor of __1.35__ by 2024. The bicycle market is booming, and there is great money to be made in this industry.

One way that companies have taken advantage of this growth is through the development of bike share programs, such as the [Metro Bike Share Program](https://bikeshare.metro.net/about/) in Los Angeles. Metro Bike Share's goal is to provide bikes for people all around the Los Angeles area to cut down on carbon emissions and get more fit. They also release all the data they collect for people to use.

**Today, we will analyze the most recent bike share data in order to determine where to expand the program and where the program needs a heavier focus on maintenance**

## Retrieve the Data
The data can be pulled from the [Metro Bike Share Program's Website](https://bikeshare.metro.net/about/data/), where we will pull the most recent data (2019 - Q3).

In [1]:
import numpy as np
import pandas as pd

### Bike Share Trip Data
First, we will pull the trip data:

In [2]:
bs_trips = pd.read_csv('../data/raw/metro-bike-share-trips-2019-q3.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There seems to be some problems with the 10th column in this file. It seems that there are multiple types in this column. We'll go ahead and import it with a generic type, and then come back to it later to figure out a more permanent solution.

In [3]:
bs_trips = pd.read_csv('../data/raw/metro-bike-share-trips-2019-q3.csv', dtype={10: np.object})
bs_trips.head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,124657107,5,7/1/2019 0:04,7/1/2019 0:09,4312,34.066990,-118.290878,4410,34.063351,-118.296799,6168,30,One Way,Monthly Pass,standard
1,124657587,9,7/1/2019 0:07,7/1/2019 0:16,3066,34.063389,-118.236160,3066,34.063389,-118.236160,17584,30,Round Trip,Monthly Pass,electric
2,124658068,5,7/1/2019 0:20,7/1/2019 0:25,4410,34.063351,-118.296799,4312,34.066990,-118.290878,18920,30,One Way,Monthly Pass,electric
3,124659747,20,7/1/2019 0:44,7/1/2019 1:04,3045,34.028511,-118.256668,4275,34.012520,-118.285896,6016,1,One Way,Walk-up,standard
4,124660227,27,7/1/2019 0:44,7/1/2019 1:11,3035,34.048401,-118.260948,3049,34.056969,-118.253593,5867,30,One Way,Monthly Pass,standard


Let's analyze the columns in this dataset

In [4]:
bs_trips.keys()

Index(['trip_id', 'duration', 'start_time', 'end_time', 'start_station',
       'start_lat', 'start_lon', 'end_station', 'end_lat', 'end_lon',
       'bike_id', 'plan_duration', 'trip_route_category', 'passholder_type',
       'bike_type'],
      dtype='object')

|Key                |Description|
|:------------------|:----------|
|trip_id            |Identifier for the trip|
|duration           |How long the trip takes (in minutes)|
|start_time         |The start time of the trip|
|end_time           |The end time of the trip|
|start_station      |Identifier for the station where the user starts their trip|
|start_lat          |Latitude of the start station|
|start_lon          |Longitude of the start station|
|end_station        |Identifier for the station where the user ends their trip|
|end_lat            |Latitude of the end station|
|end_lon            |Longitude of the end station|
|bike_id            |Identifier for the bike (keep in mind, this was our problem column from earlier|
|plan_duration      |The number of days that the plan the user is using|
|trip_route_category|"Round Trip" if the user ends in the same place they started. "One Way" otherwise|
|passholder_type    |The name of the user's plan|
|bike_type          |The type of bike that is being used|

And finally, a little extra information:

In [5]:
bs_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92124 entries, 0 to 92123
Data columns (total 15 columns):
trip_id                92124 non-null int64
duration               92124 non-null int64
start_time             92124 non-null object
end_time               92124 non-null object
start_station          92124 non-null int64
start_lat              89985 non-null float64
start_lon              89985 non-null float64
end_station            92124 non-null int64
end_lat                88052 non-null float64
end_lon                88052 non-null float64
bike_id                92124 non-null object
plan_duration          92124 non-null int64
trip_route_category    92124 non-null object
passholder_type        92124 non-null object
bike_type              92124 non-null object
dtypes: float64(4), int64(5), object(6)
memory usage: 10.5+ MB


### Station Data
Next, we'll pull the station data:

In [6]:
bs_stations = pd.read_csv('../data/raw/metro-bike-share-stations-2019-10-1.csv')
bs_stations.head()

,Station_ID,Station_Name,Go_live_date,Region,Status
0,3000,Virtual Station,7/7/2016,NaN,Active
1,3005,7th & Flower,7/7/2016,DTLA,Active
2,3006,Olive & 8th,7/7/2016,DTLA,Active
3,3007,5th & Grand,7/7/2016,DTLA,Active
4,3008,Figueroa & 9th,7/7/2016,DTLA,Active


That import went much more smoothly. Let's analyze the columns:

In [7]:
bs_stations.keys()

Index(['Station_ID', 'Station_Name', 'Go_live_date', 'Region ', 'Status'], dtype='object')

|Key         |Description|
|:-----------|:----------|
|Station_ID  |Identifier for the station|
|Station_Name|Intersection where the station is located|
|Go_live_date|When the station was first made active|
|Region      |The municipality or area where a station is located|
|Status      |"Active" if the station is still operable, "Inactive" if the station has been shut down|

And finally, a little extra information:

In [8]:
bs_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 5 columns):
Station_ID      245 non-null int64
Station_Name    245 non-null object
Go_live_date    245 non-null object
Region          244 non-null object
Status          245 non-null object
dtypes: int64(1), object(4)
memory usage: 9.6+ KB


## Cleaning Up the Data

### Trip Cleanup
Let's start by going back to our import issue with the trip data. Let's analyze column 10 (`bike_id`) in the trip dataframe:

In [9]:
bs_trips["bike_id"].describe()

count     92124
unique     2042
top       17584
freq        397
Name: bike_id, dtype: object

In [10]:
bs_trips["bike_id"].head()

0     6168
1    17584
2    18920
3     6016
4     5867
Name: bike_id, dtype: object

It looks like the values should be numerics, but there must be an invalid value to give us the error we got during import. Let's check and see if all of our values are indeed numerics.

In [11]:
bs_trips[~bs_trips["bike_id"].str.isnumeric()]

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
32309,126711706,10,8/5/2019 5:32,8/5/2019 5:42,3000,NaN,NaN,3000,NaN,NaN,Rblock,365,Round Trip,Walk-up,standard
32313,126712182,7,8/5/2019 5:43,8/5/2019 5:50,3000,NaN,NaN,3000,NaN,NaN,Rblock,365,Round Trip,Walk-up,standard


Aha! It looks like we have 2 values that we were not expecting. The `bike_id` value for these two entries above is `"Rblock"`, a string, not a numeric. This would make sense why we had trouble importing before. Looking at the information, it looks like we can just ignore these two entries because they do not provide any particularly useful information. We will keep in mind that we got rid of these later in case it comes up again.

In [12]:
indecesToDrop = bs_trips[~bs_trips["bike_id"].str.isnumeric()].index
bs_trips.drop(indecesToDrop, inplace=True)
bs_trips["bike_id"] = pd.to_numeric(bs_trips["bike_id"])

Next, let's make sure we don't have any duplicate trips. We can do this by checking the trip IDs:

In [13]:
bs_trips_value_counts = bs_trips["trip_id"].value_counts()
bs_trips_value_counts[bs_trips_value_counts > 1].empty

True

Awesome! We have no duplicate trips. Finally, let's run through the rest of our data and make sure it all looks up to code. Let's start by checking for null:

In [14]:
bs_trips.isnull().sum()

trip_id                   0
duration                  0
start_time                0
end_time                  0
start_station             0
start_lat              2137
start_lon              2137
end_station               0
end_lat                4070
end_lon                4070
bike_id                   0
plan_duration             0
trip_route_category       0
passholder_type           0
bike_type                 0
dtype: int64

Strange, we seem to be having some null values for some latitude and longitude. Let's look more closely:

In [15]:
bs_trips[bs_trips["start_lat"].isnull()].head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
287,124691541,18,7/1/2019 11:44,7/1/2019 12:02,3000,NaN,NaN,4345,33.995560,-118.481552,15969,1,One Way,Walk-up,smart
322,124692870,4,7/1/2019 12:26,7/1/2019 12:30,4285,NaN,NaN,4325,34.022518,-118.400589,15439,30,One Way,Monthly Pass,smart
382,124696875,31,7/1/2019 13:25,7/1/2019 13:56,4286,NaN,NaN,4345,33.995560,-118.481552,15696,365,One Way,Walk-up,smart
387,124696867,22,7/1/2019 13:33,7/1/2019 13:55,4286,NaN,NaN,4345,33.995560,-118.481552,15929,365,One Way,Walk-up,smart
390,124696635,14,7/1/2019 13:38,7/1/2019 13:52,4286,NaN,NaN,4345,33.995560,-118.481552,16261,365,One Way,Walk-up,smart


In [16]:
bs_trips[bs_trips["end_lat"].isnull()].head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
11,124673048,72,7/1/2019 4:23,7/1/2019 5:35,4344,34.014309,-118.491341,4286,NaN,NaN,15936,1,One Way,Walk-up,smart
40,124676879,12,7/1/2019 6:43,7/1/2019 6:55,3019,34.038609,-118.260857,3000,NaN,NaN,16303,365,One Way,Annual Pass,electric
52,124678746,34,7/1/2019 7:01,7/1/2019 7:35,4372,34.027882,-118.388901,4285,NaN,NaN,15948,365,One Way,Walk-up,smart
126,125015488,1440,7/1/2019 8:24,7/6/2019 11:13,4344,34.014309,-118.491341,3000,NaN,NaN,15883,1,One Way,Walk-up,smart
165,124682922,4,7/1/2019 9:03,7/1/2019 9:07,3058,34.035801,-118.233170,3000,NaN,NaN,12205,30,One Way,Monthly Pass,standard


Well, due to these missing latitude and longitude coordinates, we can't rely on them to locate our bike stations. Thankfully, we still have the station identifiers, which we can use in conjunction with our stations dataframe to locate the stations. We will go ahead and drop the latitude and longitude columns so as to remove any confusion:

In [17]:
bs_trips = bs_trips.drop(columns=["start_lat", "start_lon", "end_lat", "end_lon"])
bs_trips.head()

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,124657107,5,7/1/2019 0:04,7/1/2019 0:09,4312,4410,6168,30,One Way,Monthly Pass,standard
1,124657587,9,7/1/2019 0:07,7/1/2019 0:16,3066,3066,17584,30,Round Trip,Monthly Pass,electric
2,124658068,5,7/1/2019 0:20,7/1/2019 0:25,4410,4312,18920,30,One Way,Monthly Pass,electric
3,124659747,20,7/1/2019 0:44,7/1/2019 1:04,3045,4275,6016,1,One Way,Walk-up,standard
4,124660227,27,7/1/2019 0:44,7/1/2019 1:11,3035,3049,5867,30,One Way,Monthly Pass,standard


Finally, we will convert the start and end times to date objects as their dtype is currently just an object. We first need to know the format that they are saved in:

In [18]:
bs_trips["start_time"].tail(10)

92114    9/30/2019 23:03
92115    9/30/2019 23:04
92116    9/30/2019 23:08
92117    9/30/2019 23:15
92118    9/30/2019 23:20
92119    9/30/2019 23:32
92120    9/30/2019 23:38
92121    9/30/2019 23:40
92122    9/30/2019 23:41
92123    9/30/2019 23:58
Name: start_time, dtype: object

Using `9/30/2019 23:58` as a guide, we can see that the format is "*month/day/year hour(24):minute*". Now we can convert the data:

In [21]:
bs_trips["start_time"] = pd.to_datetime(bs_trips["start_time"], format="%m/%d/%Y %H:%M")
bs_trips["start_time"].describe()

count                   92122
unique                  57272
top       2019-08-24 18:50:00
freq                        9
first     2019-07-01 00:04:00
last      2019-09-30 23:58:00
Name: start_time, dtype: object

In [22]:
bs_trips["end_time"] = pd.to_datetime(bs_trips["end_time"], format="%m/%d/%Y %H:%M")
bs_trips["end_time"].describe()

count                   92122
unique                  55656
top       2019-09-27 08:28:00
freq                       16
first     2019-07-01 00:09:00
last      2019-10-02 12:04:00
Name: end_time, dtype: object

### Stations Cleanup
There's one quick issue with the stations dataframe that we should fix first. There is a typo in the key name of the `Region` column. Let's fix that real quick:

In [23]:
bs_stations = bs_stations.rename(columns={"Region ": "Region"})

While we're here, let's go ahead and change all the key names to lowercase to remain consistent across dataframes:

In [24]:
bs_stations.columns = [col.lower() for col in bs_stations]
bs_stations.keys()

Index(['station_id', 'station_name', 'go_live_date', 'region', 'status'], dtype='object')

Next, let's check for any null values in this dataframe:

In [25]:
bs_stations.isnull().sum()

station_id      0
station_name    0
go_live_date    0
region          1
status          0
dtype: int64

We seem to have one null region. Let's investigate:

In [26]:
bs_stations[bs_stations["region"].isnull()]

,station_id,station_name,go_live_date,region,status
0,3000,Virtual Station,2016-07-07,NaN,Active


According to the Metro Bike Share website...
> "Virtual Station" is used by staff to check in or check out a bike remotely for a special event or in a situation in which a bike could not otherwise be checked in or out to a station.

With this in mind, we're going to want to drop all trips with the station 3000, as we are only interested in the public use of the bike share, rather than staff use. We will wait until we combine the dataframes to do the drop.

Finally, we will do what we did earlier with `bs_trips["start_time"]` and `bs_trips["end_time"]` and convert the data in `go_live_date` to a date object. Let's find the format:

In [27]:
bs_stations["go_live_date"].tail(10)

235   2019-09-11
236   2019-09-11
237   2019-09-12
238   2019-09-12
239   2019-09-26
240   2019-09-18
241   2019-10-02
242   2019-10-02
243   2019-09-25
244   2019-09-25
Name: go_live_date, dtype: datetime64[ns]

Using `9/25/2019` as a guide, we can see that the format is "*month/day/year*". Now we can convert the data:

In [29]:
bs_stations["go_live_date"] = pd.to_datetime(bs_stations["go_live_date"], format="%m/%d/%Y")
bs_stations["go_live_date"].describe()

count                     245
unique                     67
top       2016-07-07 00:00:00
freq                       55
first     2016-07-07 00:00:00
last      2019-10-02 00:00:00
Name: go_live_date, dtype: object

## Filtering the Data

For the trips dataframe, we can ignore `duration`, `bike_id`, `plan_duration`, `trip_route_category`, `passholder_type`, and `bike_type` columns for what we are attempting to accomplish. Let's keep the `trip_id`, `start_time`, and `end_time` in case we decide to check for when they are being used, as this can be very useful in figuring out what areas need growth. The `start_station` and `end_station` identifiers will be necessary for merging.

In [45]:
bs_trips = bs_trips.drop(columns=["duration", "bike_id", "plan_duration", "trip_route_category", "passholder_type",
                                  "bike_type"])
bs_trips.head()

,trip_id,start_time,end_time,start_station,end_station
0,124657107,2019-07-01 00:04:00,2019-07-01 00:09:00,4312,4410
1,124657587,2019-07-01 00:07:00,2019-07-01 00:16:00,3066,3066
2,124658068,2019-07-01 00:20:00,2019-07-01 00:25:00,4410,4312
3,124659747,2019-07-01 00:44:00,2019-07-01 01:04:00,3045,4275
4,124660227,2019-07-01 00:44:00,2019-07-01 01:11:00,3035,3049


We can keep most of the columns in the stations dataframe, as they can almost all be useful in our analysis. The only one that may not be as useful is the `go_live_date`. The only way this will be useful is if there are some stations that were built before we started tracking the trips in this quarter. Let's find this out:

In [46]:
bs_trips["start_time"].describe()

count                   92122
unique                  57272
top       2019-08-24 18:50:00
freq                        9
first     2019-07-01 00:04:00
last      2019-09-30 23:58:00
Name: start_time, dtype: object

The first date that we start tracking is **July 1st, 2019** (`2019-07-01`). Let's check the last value of `go_live_date`. If they overlap, we know we must keep `go_live_date` in mind when analyzing our data:

In [47]:
bs_stations["go_live_date"].describe()

count                     245
unique                     67
top       2016-07-07 00:00:00
freq                       55
first     2016-07-07 00:00:00
last      2019-10-02 00:00:00
Name: go_live_date, dtype: object

According to this, the last station to be entered in the stations dataframe went live on October 2nd, 2019 (`2019-10-02`). Because this is after the first trip logged in the trips dataframe, we must keep `go_live_date` in mind when analyzing our data.

The rest of the columns are useful as well. `station_id` will be determine how we merge the dataframes, `station_name` will be useful in finding where the stations are, `region` will be useful for analysis of stations, and `status` is important to know what work and what don't.

## Merging the Data

Now it is time to merge our data! We will merge the stations data in for the start station *and* the end station.

In [56]:
# Merge the dataframes
bs_combined = bs_trips.merge(bs_stations, left_on="start_station", right_on="station_id").drop(columns=["station_id"])\
    .merge(bs_stations, left_on="end_station", right_on="station_id", suffixes=["_start", "_end"])\
    .drop(columns=["station_id"])
# Rename the columns
bs_combined = bs_combined.rename(columns={
    "station_name_start": "start_station_name",
    "go_live_date_start": "start_station_go_live_date",
    "region_start": "start_station_region",
    "status_start": "start_station_status",
    "station_name_end": "end_station_name",
    "go_live_date_end": "end_station_go_live_date",
    "region_end": "end_station_region",
    "status_end": "end_station_status",
})
# Move the columns
cols = bs_combined.keys().tolist()
cols.insert(cols.index('start_station_status'), cols.pop(cols.index('end_station')))
cols.insert(cols.index('start_station_status'), cols.pop(cols.index('end_time')))
bs_combined = bs_combined.reindex(columns=cols)

bs_combined.head()

,trip_id,start_time,start_station,start_station_name,start_station_go_live_date,start_station_region,start_station_status,end_time,end_station,end_station_name,end_station_go_live_date,end_station_region,end_station_status
0,124657107,2019-07-01 00:04:00,4312,4th & Vermont,2019-02-07,DTLA,Active,2019-07-01 00:09:00,4410,Kenmore & 6th,2019-06-04,DTLA,Active
1,124754118,2019-07-02 13:12:00,4312,4th & Vermont,2019-02-07,DTLA,Active,2019-07-02 13:15:00,4410,Kenmore & 6th,2019-06-04,DTLA,Active
2,124877230,2019-07-04 15:20:00,4312,4th & Vermont,2019-02-07,DTLA,Active,2019-07-04 15:24:00,4410,Kenmore & 6th,2019-06-04,DTLA,Active
3,124900108,2019-07-05 00:08:00,4312,4th & Vermont,2019-02-07,DTLA,Active,2019-07-05 00:12:00,4410,Kenmore & 6th,2019-06-04,DTLA,Active
4,125000409,2019-07-06 18:06:00,4312,4th & Vermont,2019-02-07,DTLA,Active,2019-07-06 18:11:00,4410,Kenmore & 6th,2019-06-04,DTLA,Active


## Analyzing the Data
Time for the fun part! Let's analyze some data